In [51]:
import gensim
import import_ipynb
import itertools
import nltk
import NLP_preprocessing as pre
import numpy as np
import pandas as pd
import tensorflow as tf
import time

from gensim import corpora
from keras.callbacks import TensorBoard, EarlyStopping  
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Flatten
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

#### Load data

In [2]:
data = pd.read_excel('отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
max_words = 20000
max_len = 78
num_classes = 5

epochs = 20
batch_size = 16
print_batch_n = 100

#### Preprocessing

In [4]:
%%time
tqdm.pandas()
data['Content_pre'] = data['Content'].progress_apply(pre.text_preprocess_light)
data.insert(1, 'Content_token', [pre.word_tokenize(text) for text in data['Content_pre']], allow_duplicates=True)
stem_content_token =[]
for tokens in tqdm(data['Content_token']):
    stem_token =  [pre.ps().stem(token) for token in tokens]
    stem_content_token.append(stem_token)
data.insert(2, 'Content_stemmed', stem_content_token, allow_duplicates=True)
lemma_content_token =[]
for tokens in tqdm(data['Content_stemmed']):
    lemma_token =  [pre.lem().lemmatize(token) for token in tokens]
    lemma_content_token.append(lemma_token)
data.insert(3, 'Content_lemmatized', lemma_content_token, allow_duplicates=True)

100%|██████████████████████████████████████████████████████████████████████████| 20659/20659 [00:02<00:00, 9951.36it/s]

Wall time: 27.8 s


In [5]:
data

,Rating,Content_token,Content_stemmed,Content_lemmatized,Content,Date,Content_pre
0,5,[],[],[],It just works!,2017-08-14,
1,4,"[целое, удобноной, приложение, минус, хотеть, ...","[целое, удобноной, приложение, минус, хотеть, ...","[целое, удобноной, приложение, минус, хотеть, ...",В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложение минус хотеть большо...
2,5,[отлично],[отлично],[отлично],Отлично все,2017-08-14,отлично
3,5,"[зависать, работа, антивирус, ранее, пользоват...","[зависать, работа, антивирус, ранее, пользоват...","[зависать, работа, антивирус, ранее, пользоват...",Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать работа антивирус ранее пользоваться н...
4,5,"[удобно, работать, быстро]","[удобно, работать, быстро]","[удобно, работать, быстро]","Очень удобно, работает быстро.",2017-08-14,удобно работать быстро
...,...,...,...,...,...,...,...
20654,1,"[шляпа, роот, право, бесполезный, прога, разра...","[шляпа, роот, право, бесполезный, прога, разра...","[шляпа, роот, право, бесполезный, прога, разра...","Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,шляпа роот право бесполезный прога разраб охре...
20655,5,[],[],[],Ок,2017-06-01,
20656,4,[довольный],[довольный],[довольный],Доволен,2017-06-01,довольный
20657,1,"[песопаснастя, рута, нужын]","[песопаснастя, рута, нужын]","[песопаснастя, рута, нужын]","Песопаснасть, рут ни нужын",2017-06-01,песопаснастя рута нужын


In [6]:
# Словарь уникальных токенов
dictionary = corpora.Dictionary(data['Content_lemmatized'])

In [7]:
# Выкидывание редких слов
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('content.dict')
len(dictionary.keys())

1101

In [8]:
tokens = [dictionary[i] for i in range(len(dictionary))]
tokens

['большой',
 'данные',
 'доступ',
 'минус',
 'ограниченный',
 'пользоваться',
 'приложение',
 'режим',
 'хотеть',
 'целое',
 'отлично',
 'антивирус',
 'зависать',
 'нормально',
 'работа',
 'ранее',
 'быстро',
 'работать',
 'удобно',
 'норма',
 'удобный',
 'устраивать',
 'банкомат',
 'вечно',
 'отличие',
 'тупить',
 'чётко',
 'нельзя',
 'уведомление',
 'удалять',
 'звонок',
 'книга',
 'один',
 'слово',
 'замечательный',
 'штука',
 'отличный',
 'практично',
 'оплата',
 'перевод',
 'платёж',
 'вообще',
 'карта',
 'пополнять',
 'жаль',
 'история',
 'хороший',
 'лишний',
 'полезный',
 'висеть',
 'дважды',
 'загрузка',
 'заход',
 'пароль',
 'первый',
 'список',
 'требовать',
 'запуститься',
 'сбербанк',
 'часы',
 'супер',
 'комфортно',
 'поиск',
 'служба',
 'молодец',
 'круто',
 'взгляд',
 'нарекание',
 'недолго',
 'мнеудобный',
 'ура',
 'долгий',
 'идти',
 'использование',
 'исправт',
 'конец',
 'неустанавливаться',
 'обновление',
 'повторный',
 'попытка',
 'последний',
 'программа',
 'скач

In [9]:
voc = {v: k for k, v in dict(enumerate(tokens, 1)).items()}

In [10]:
def text_to_sequence(tokens, maxlen):
    result = []
    for word in tokens:
        if word in voc:
            result.append(voc[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [11]:
data_train = np.asarray([text_to_sequence(text, max_len) for text in data['Content_lemmatized']], dtype=np.int32)
data_train

array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    6,    5,    8],
       [   0,    0,    0, ...,    0,    0,   11],
       ...,
       [   0,    0,    0, ...,    0,    0,  123],
       [   0,    0,    0, ...,    0,    0,  294],
       [   0,    0,    0, ..., 1056,  210, 1024]])

In [12]:
count = 0
max_count = 0
for i in range(len(data_train)):
    for j in range(len(data_train[i])):
        if data_train[i][j] != 0:
            count += 1
        if max_count < count:
            max_count = count
    count = 0
print(max_count)

77


In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data.Rating, test_size=0.3, random_state=1)

In [14]:
X_train.shape

(14461, 78)

In [15]:
le = LabelEncoder()

y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)

le.classes_

array([1, 2, 3, 4, 5], dtype=int64)

In [16]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

##### Gpu acceleration and reduced memory usage

In [17]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.config.set_soft_device_placement(True)

#### Model with default Embedding layer

In [27]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128, 
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dropout(rate=0.25))
model.add(Flatten())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [28]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=2,
          verbose=1,
          validation_split=0.1)

Epoch 1/2
814/814 [==============================] - 14s 16ms/step - loss: 0.7254 - accuracy: 0.7619 - val_loss: 0.6575 - val_accuracy: 0.7754
Epoch 2/2
814/814 [==============================] - 14s 17ms/step - loss: 0.6278 - accuracy: 0.7806 - val_loss: 0.6564 - val_accuracy: 0.7768


In [30]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

388/388 [==============================] - 2s 5ms/step - loss: 0.6679 - accuracy: 0.7656
Test score: 0.6679004430770874
Test accuracy: 0.7655695676803589


#### Model with pre-trained Embedding layer (tayga_upos_skipgram_300_2_2019)

http://vectors.nlpl.eu/repository/20/185.zip

In [36]:
#Загрузим предобученные векторы
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('tayga_1_2.vec').vectors 

In [39]:
len(word_vectors[0])

300

In [46]:
#обрежем word_vectors по размеру эмбеддингов предыдущей задачи
word_vectors_matrix = [word_vectors[i][:128] for i in range(max_words)]
word_vectors_matrix

[array([ 6.80980906e-02, -6.40336308e-04,  2.87309550e-02,  6.37063012e-02,
         7.53745157e-03,  1.13257207e-02,  5.33963665e-02, -1.30230915e-02,
        -3.50625552e-02,  1.23897836e-01, -8.27610958e-04, -4.30137757e-03,
        -2.49322634e-02, -5.57406880e-02,  6.21645041e-02,  2.34909896e-02,
        -3.39659899e-02,  4.93191518e-02, -4.97348756e-02,  6.10462129e-02,
        -6.48630857e-02, -2.91896854e-02,  3.97506543e-02,  5.05782254e-02,
        -1.20987140e-01, -9.06800181e-02,  8.81398618e-02,  5.23627810e-02,
         4.45564883e-03, -4.31095436e-02,  7.27595203e-03,  2.89805923e-02,
         3.53384094e-04,  1.02831215e-01, -6.52962327e-02,  3.36193331e-02,
         5.91043308e-02, -3.51957604e-02, -2.60913316e-02,  5.49497036e-03,
         8.98569488e-05,  5.36094978e-02,  3.24335359e-02, -2.17619929e-02,
        -6.57182112e-02, -3.86023298e-02, -2.75710765e-02,  3.76490615e-02,
         7.30575854e-03,  4.49738055e-02, -1.11787599e-02, -1.80596602e-03,
        -4.1

In [47]:
#инициализируем веса в эмбеддинге
initializer = tf.keras.initializers.Constant(word_vectors_matrix)

In [48]:
model = Sequential()

model.add(Embedding(input_dim=max_words, 
                    output_dim=128,
                    embeddings_initializer=initializer,
                    input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dropout(rate=0.25))
model.add(Flatten())

model.add(Dense(10))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [49]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [53]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 4s 92ms/step - loss: 1.2245 - accuracy: 0.6773 - val_loss: 0.9994 - val_accuracy: 0.7091
Epoch 2/20
26/26 [==============================] - 2s 65ms/step - loss: 0.8591 - accuracy: 0.7059 - val_loss: 0.7578 - val_accuracy: 0.7091


In [54]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 23ms/step - loss: 0.7589 - accuracy: 0.7073
Test score: 0.7589393854141235
Test accuracy: 0.7073249220848083


#### Вывод:
Лучший результат у непредобученной модели.